In [45]:
import tensorflow as tf   
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random 

In [47]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [48]:
import sys
print(sys.executable)

c:\Users\dangn\anaconda3\envs\tensorflowgpu\python.exe


In [49]:
import tensorflow as tf   
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [50]:
df = pd.read_csv("datasets\covid19vaccinesbycountybydemographic.csv")

In [51]:
df = df[df['demographic_category'].str.contains("Race/Ethnicity")]

df['demographic_category'].unique()

array(['Race/Ethnicity'], dtype=object)

In [52]:
df['demographic_value'].unique()

array(['American Indian or Alaska Native', 'Asian',
       'Black or African American', 'Latino', 'Multiracial',
       'Native Hawaiian or Other Pacific Islander', 'Other Race',
       'Unknown', 'White'], dtype=object)

In [53]:
df = df[~df['demographic_value'].str.contains("Other Race|Unknown")]

df['demographic_value'].unique()

array(['American Indian or Alaska Native', 'Asian',
       'Black or African American', 'Latino', 'Multiracial',
       'Native Hawaiian or Other Pacific Islander', 'White'], dtype=object)

In [54]:
df['county'].unique()

array(['Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa',
       'Contra Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn',
       'Humboldt', 'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 'Lassen',
       'Los Angeles', 'Madera', 'Marin', 'Mariposa', 'Mendocino',
       'Merced', 'Modoc', 'Mono', 'Monterey', 'Napa', 'Nevada', 'Orange',
       'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito',
       'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin',
       'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara',
       'Santa Cruz', 'Shasta', 'Sierra', 'Siskiyou', 'Solano', 'Sonoma',
       'Stanislaus', 'Statewide', 'Sutter', 'Tehama', 'Trinity', 'Tulare',
       'Tuolumne', 'Ventura', 'Yolo', 'Yuba'], dtype=object)

In [55]:
encoder = LabelEncoder()
df['county_encoded'] = encoder.fit_transform(df['county'])
df['demographic_encoded'] = encoder.fit_transform(df['demographic_value'])

In [56]:
df['percent_fully_vaccinated'] = df['cumulative_fully_vaccinated'] / df['est_population']

df = df[df['percent_fully_vaccinated'] != 0]
df = df[df['percent_fully_vaccinated'].notnull()]
df['percent_fully_vaccinated'] = df['percent_fully_vaccinated'].clip(upper=1)

In [57]:
df['administered_date'] = pd.to_datetime(df['administered_date'], format = '%Y-%m-%d')

df['time_since_start'] = (df['administered_date'] - df['administered_date'].min()).dt.days

In [58]:

prep_df = df[['percent_fully_vaccinated', 'county_encoded', 'demographic_encoded', 'time_since_start']]

prep_df

,percent_fully_vaccinated,county_encoded,demographic_encoded,time_since_start
24240,1.000000,0,0,1465
24241,1.000000,0,0,1464
24242,1.000000,0,0,1463
24243,1.000000,0,0,1462
24244,1.000000,0,0,1461
...,...,...,...,...
2179837,0.000025,58,6,20
2179838,0.000025,58,6,19
2179839,0.000025,58,6,18
2179840,0.000025,58,6,17


In [59]:
prep_df = prep_df[prep_df['percent_fully_vaccinated'] != 0]
prep_df = prep_df[prep_df['percent_fully_vaccinated'].notnull()]


prep_df

,percent_fully_vaccinated,county_encoded,demographic_encoded,time_since_start
24240,1.000000,0,0,1465
24241,1.000000,0,0,1464
24242,1.000000,0,0,1463
24243,1.000000,0,0,1462
24244,1.000000,0,0,1461
...,...,...,...,...
2179837,0.000025,58,6,20
2179838,0.000025,58,6,19
2179839,0.000025,58,6,18
2179840,0.000025,58,6,17


In [60]:
time_step = 10


grouped_county = prep_df.groupby(['county_encoded', 'demographic_encoded'])


In [61]:
X = []
y = []

for name, group in grouped_county:
    values = group[['percent_fully_vaccinated', 'county_encoded', 'demographic_encoded', 'time_since_start']].values

    for i in range(len(values) - time_step):
        X.append(values[i:i+time_step,:])
        y.append(values[i+time_step, 0])

X = np.array(X)
y = np.array(y)

In [62]:
X.shape

(518314, 10, 4)

In [63]:
np.set_printoptions(suppress=True)

X

array([[[   1.        ,    0.        ,    0.        , 1465.        ],
        [   1.        ,    0.        ,    0.        , 1464.        ],
        [   1.        ,    0.        ,    0.        , 1463.        ],
        ...,
        [   1.        ,    0.        ,    0.        , 1458.        ],
        [   1.        ,    0.        ,    0.        , 1457.        ],
        [   1.        ,    0.        ,    0.        , 1456.        ]],

       [[   1.        ,    0.        ,    0.        , 1464.        ],
        [   1.        ,    0.        ,    0.        , 1463.        ],
        [   1.        ,    0.        ,    0.        , 1462.        ],
        ...,
        [   1.        ,    0.        ,    0.        , 1457.        ],
        [   1.        ,    0.        ,    0.        , 1456.        ],
        [   1.        ,    0.        ,    0.        , 1455.        ]],

       [[   1.        ,    0.        ,    0.        , 1463.        ],
        [   1.        ,    0.        ,    0.        , 1462. 

In [64]:
model = Sequential()

model.add(LSTM(20, activation = "relu", input_shape = (X.shape[1], X.shape[2])))
model.add(Dense(10))
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 20)                2000      
                                                                 
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2,221
Trainable params: 2,221
Non-trainable params: 0
_________________________________________________________________


In [65]:
history = model.fit(X, y, epochs = 100, batch_size=32, validation_split=0.2)

Epoch 1/100
12958/12958 [==============================] - 227s 17ms/step - loss: 4.8557 - val_loss: 7.1067e-04
Epoch 2/100
12958/12958 [==============================] - 218s 17ms/step - loss: 3.6878e-04 - val_loss: 4.0636e-05
Epoch 3/100
12958/12958 [==============================] - 234s 18ms/step - loss: 2.5543e-04 - val_loss: 5.1429e-05
Epoch 4/100
12958/12958 [==============================] - 231s 18ms/step - loss: 2.0840e-04 - val_loss: 3.0023e-04
Epoch 5/100
12958/12958 [==============================] - 231s 18ms/step - loss: 1.6571e-04 - val_loss: 1.9668e-04
Epoch 6/100
12958/12958 [==============================] - 231s 18ms/step - loss: 1.4878e-04 - val_loss: 2.4209e-05
Epoch 7/100
12958/12958 [==============================] - 300s 23ms/step - loss: 1.7676e-04 - val_loss: 4.2434e-04
Epoch 8/100
12958/12958 [==============================] - 310s 24ms/step - loss: 2.9347e-04 - val_loss: 2.8491e-04
Epoch 9/100
12958/12958 [==============================] - 305s 23ms/step - 

In [68]:
model.save('models/ethnicity_county', save_format='tf')


INFO:tensorflow:Assets written to: models/ethnicity_county\assets


In [67]:
model = load_model("models\ethnicity_county")

OSError: No file or directory found at models\ethnicity_county

In [ ]:
encoded_county = list(zip(df['county_encoded'].unique(), df['county'].unique()))

sorted(encoded_county, key=lambda x: x[0])

[(0, 'Alameda'),
 (1, 'Alpine'),
 (2, 'Amador'),
 (3, 'Butte'),
 (4, 'Calaveras'),
 (5, 'Colusa'),
 (6, 'Contra Costa'),
 (7, 'Del Norte'),
 (8, 'El Dorado'),
 (9, 'Fresno'),
 (10, 'Glenn'),
 (11, 'Humboldt'),
 (12, 'Imperial'),
 (13, 'Inyo'),
 (14, 'Kern'),
 (15, 'Kings'),
 (16, 'Lake'),
 (17, 'Lassen'),
 (18, 'Los Angeles'),
 (19, 'Madera'),
 (20, 'Marin'),
 (21, 'Mariposa'),
 (22, 'Mendocino'),
 (23, 'Merced'),
 (24, 'Modoc'),
 (25, 'Mono'),
 (26, 'Monterey'),
 (27, 'Napa'),
 (28, 'Nevada'),
 (29, 'Orange'),
 (30, 'Placer'),
 (31, 'Plumas'),
 (32, 'Riverside'),
 (33, 'Sacramento'),
 (34, 'San Benito'),
 (35, 'San Bernardino'),
 (36, 'San Diego'),
 (37, 'San Francisco'),
 (38, 'San Joaquin'),
 (39, 'San Luis Obispo'),
 (40, 'San Mateo'),
 (41, 'Santa Barbara'),
 (42, 'Santa Clara'),
 (43, 'Santa Cruz'),
 (44, 'Shasta'),
 (45, 'Sierra'),
 (46, 'Siskiyou'),
 (47, 'Solano'),
 (48, 'Sonoma'),
 (49, 'Stanislaus'),
 (50, 'Statewide'),
 (51, 'Sutter'),
 (52, 'Tehama'),
 (53, 'Trinity'),
 (5

In [ ]:
encoded_ethnicity = list(zip(df['demographic_value'].unique(), df['demographic_encoded'].unique()))

sorted(encoded_ethnicity, key=lambda x: x[0])

[('American Indian or Alaska Native', 0),
 ('Asian', 1),
 ('Black or African American', 2),
 ('Latino', 3),
 ('Multiracial', 4),
 ('Native Hawaiian or Other Pacific Islander', 5),
 ('White', 6)]

In [ ]:
#forecast - 30 days - Santa Clara - Asian
forecast = 30

forecast_df = prep_df[(prep_df['county_encoded'] == 42) & (prep_df['demographic_encoded'] == 1)]

forecast_df

,percent_fully,county_encoded,demographic_encoded,time_since_start
1577103,2.966620e-01,42,1,1488
1577104,2.966620e-01,42,1,1487
1577105,2.966620e-01,42,1,1486
1577106,2.966620e-01,42,1,1485
1577107,2.966620e-01,42,1,1484
...,...,...,...,...
1578552,1.382155e-06,42,1,39
1578553,9.214365e-07,42,1,38
1578554,9.214365e-07,42,1,37
1578555,9.214365e-07,42,1,36
